In [1]:
import itertools as it

import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation


In [2]:

def box3d(n: int=16) -> np.ndarray:

    points = []
    N = tuple(np.linspace(-1, 1, n))
    
    for i, j in [(-1, -1), (-1, 1), (1, 1), (0, 0)]:
        points.extend(set(it.permutations([(i, ) * n, (j, ) * n, N])))
    
    return np.hstack(points) / 2


def Pi(x: np.ndarray) -> np.ndarray:
    return np.vstack(x, np.ones(x.shape[1]))


def InvPi(x: np.ndarray) -> np.ndarray:
    return x[:-1] / x[-1]


def projectpoints(K: np.ndarray, R: np.ndarray, t: np.ndarray, Q: np.ndarray) -> np.ndarray:
    return K @ (R @ Q + t[:, None])


def point2line_distance2d(l: np.array, p: np.array) -> float:
    """Homogeneous 2D point to line distance"""
    return abs(l @ p) / (abs(p[2]) * np.linalg.norm(l[:2]))

In [3]:
K = np.eye(3)
R = lambda theta: np.array([[np.cos(theta), 0, np.sin(theta)],
                            [0, 1, 0], 
                            [-np.sin(theta), 0, np.cos(theta)]])
t = np.zeros(3)
t[2] = 4
Q = box3d()



In [14]:
l = np.array([1 / np.sqrt(2), 1 / np.sqrt(2), -1])
p1 = np.array([0, 0, 1])
p2 = np.array([np.sqrt(2), np.sqrt(2), 1])
p3 = np.array([np.sqrt(2), np.sqrt(2), 4])



dis(np.array([1,2,2]), np.array([2,4,3]))

2.3851391759997758

In [15]:
cv2.Rodrigues(np.array([-0.1, 0.1, -0.2]))[0]

array([[ 0.97512475,  0.19303094,  0.1089531 ],
       [-0.20298104,  0.97512475,  0.0890529 ],
       [-0.0890529 , -0.1089531 ,  0.9900499 ]])